# Calibration run wflow_sbm

In [ ]:
import os
import subprocess
import pandas as pd
import xarray as xr

from glob import glob
from pathlib import Path
from pathos.threading import ThreadPool as Pool

# Set Paths

In [ ]:
# Set Paths
ROOT = Path('/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/')
MODELS = Path(f'{ROOT}/wflow/data/')

julia_path = '/gpfs/home6/jaerts/julia-1.7.3/bin/julia'

# Config

In [ ]:
# Get available basin IDs wflow_sbm
basin_dirs = glob(f'{MODELS}/*')
basin_ids = [s.split('/')[-1] for s in basin_dirs]
basin_ids.sort()

# Set calibration interval
calibration_values = [1,5,10,15,20,
                      25,30,35,40,45,
                      50,55,60,65,70,
                      75,80,85,90,95,
                      100,125,150,175,
                      200,225,250,275,
                      300,350,400,450,550,
                      600,650,700,750,800,
                      850,900,950,1000,1500,
                      2000,2500,3000,4000,
                      4500,5000,7500,10000]
# Set available cores
cores_available = 60

# Sort basins by size 

In [ ]:
# Sort by basin size
def sort_basin_ids_by_size(basin_ids):
    sizes = []
    for basin_id in basin_ids:
        size = os.path.getsize(f'{MODELS}/{basin_id}/staticmaps.nc')
        sizes.append(size)

    df = pd.DataFrame()
    df['basin_id'] = basin_ids
    df['size'] = sizes
    df = df.sort_values('size')

    basin_ids = df.basin_id.to_list()
    
    return basin_ids

basin_ids_sorted = sort_basin_ids_by_size(basin_ids)

# Model Run functions

In [ ]:
def wflow_runner_calibration(julia_path, basin_id, calibration_value):
    print(f'Starting: {basin_id} ksathorfrac {calibration_value}')
    
    # Set config_file
    config_file = f'{MODELS}/{basin_id}/wflow_sbm_ksathorfrac_{calibration_value}.toml'
    
    # Call wflow julia command line
    subprocess.call(
                    f'{julia_path} -e "using Wflow; Wflow.run()" {config_file}',
                    # stdout=subprocess.DEVNULL,
                    # stderr=subprocess.STDOUT,
                    shell=True
                   )
    
    return print(f'Finished: {basin_id} ksathorfrac {calibration_value}')
    
    

In [ ]:
def parallel_run(julia_path, basin_ids, calibration_values, threads=cores_available):
    
    # Set number of threads (cores) used for parallel run and map threads
    if threads is None:
        pool = Pool()
    else:
        pool = Pool(nodes=threads)
        
    # Run parallel models
    pool.map(wflow_runner_calibration, julia_paths, basin_ids, calibration_values)
    return

## Check completed runs

In [ ]:
df = pd.DataFrame()
basins = []
exists = []

for basin_id in basin_ids:
    basins.append(basin_id)

    # check if file exists
    sim_file = Path(f'{MODELS}/{basin_id}/ksathorfrac_1/output.csv')
    if sim_file.is_file() is False:
        exists.append(False)
    else:
        df_sim = pd.read_csv(sim_file)
    
        # Check if csv containes output
        if len(df_sim) < 3200:
            exists.append(False)
        else:
            exists.append(True)
        
df['basin_id'] = basins
df['completed'] = exists
df = df.reset_index()
df = df[df['completed'] == False]

basin_ids_sorted = df.basin_id.to_list()
basin_ids_sorted

# Create lists and run function

In [ ]:
for basin_id in basins_redo:
    julia_paths = [julia_path] * len(calibration_values)
    basin_ids = [basin_id] * len(calibration_values)

    parallel_run(julia_path, basin_ids, calibration_values, threads=cores_available)